## Imports and Setup

In [1]:
#@Imports
import sys
import os
path = os.path.abspath('../..')
if path not in sys.path:
  sys.path.insert(0, path)

%load_ext autoreload

import collections
import concurrent.futures
import datetime
import json
import termcolor
import re

import numpy as np

from IPython import display

from concordia.agents import basic_agent
from concordia.components import agent as components
from concordia import components as generic_components
from concordia.associative_memory import associative_memory
from concordia.associative_memory import blank_memories
from concordia.associative_memory import formative_memories
from concordia.associative_memory import importance_function
from concordia.clocks import game_clock
from concordia.components import game_master as gm_components
from concordia.environment import game_master
from concordia.metrics import goal_achievement
from concordia.metrics import common_sense_morality
from concordia.metrics import opinion_of_others
from concordia.typing import agent as simulacrum_agent
from concordia.utils import html as html_lib
from concordia.utils import measurements as measurements_lib
from concordia.utils import plotting

from examples.custom_components.setup import model_setup, measurement_setup

In [2]:
#@Model Setup
model, embedder = model_setup('llama3:70b', local_models=True, streaming=False)
measurements, clock = measurement_setup(
  SETUP_TIME=datetime.datetime(hour=21, year=2024, month=10, day=1), 
  time_step = datetime.timedelta(minutes=5))

#@title Importance models
importance_model = importance_function.ConstantImportanceModel()
importance_model_gm = importance_function.ConstantImportanceModel()

## Scenario Details

In [3]:
#@Shared Memories

# The generic context will be used for the NPC context. It reflects general
# knowledge and is possessed by all characters.
scenario_premise = [
  (
    'It is 9pm on a weeknight. ' +
    'Alice returns late from work and finds her friend Dorothy at her doorstep. '
  ),
]
shared_context = model.sample_text(
  'Summarize the following passage in a concise and insightful fashion:\n'
  + '\n'.join(scenario_premise)
  + '\n'
  + 'Summary:'
)

#@title Creating character backgrounds, goals and traits. Modify to explore how it influences the outcomes

player_configs = [
  
  # Dorothy
  formative_memories.AgentConfig(
    name='Dorothy',
    gender='female',
    goal=(
      'Dorothy has to move out of her house tonight and needs someone to help her move. ' +
      "Dorothy had planned on Bob helping her move, but Bob didn't show up. " 
    ),
    context=shared_context,
    traits='responsibility: medium; aggression: medium',
    extras={
      'memory_path': f"./data/Dorothy.pkl"
    }
  ),

  # Alice
  formative_memories.AgentConfig(
    name='Alice',
    gender='female',
    goal='Alice is exhausted after work, and she needs to help her grandmother all day the next day.',
    context=shared_context,
    traits='responsibility: medium; aggression: medium',
    extras={
      'memory_path': f"./data/Alice.pkl"
    }
  )
]

#@Memory factories
blank_memory_factory = blank_memories.MemoryFactory(
  model=model,
  embedder=embedder,
  importance=importance_model.importance,
  clock_now=clock.now,
)

formative_memory_factory = formative_memories.FormativeMemoryFactory(
  model=model,
  shared_memories=scenario_premise,
  blank_memory_factory_call=blank_memory_factory.make_blank_memory,
)

Alice comes home from work late one weekday evening to find her friend Dorothy waiting for her at her doorstep.

## Agent Structure

In [4]:
#@Define agent structure
%autoreload 2
from examples.custom_components import tpb_llama3 as tpb
from examples.custom_components import utils

def build_agent(player_config: formative_memories.AgentConfig) -> basic_agent.BasicAgent:

  if hasattr(player_config, 'extras'):
    memory_path = player_config.extras['memory_path'] if 'memory_path' in player_config.extras.keys() else None
  else:
    memory_path = None

  if memory_path is None:
    mem = formative_memory_factory.make_memories(player_config)
  else:
    mem = utils.load_memories(blank_memory_factory.make_blank_memory(), memory_path)

  observation = components.observation.Observation(
    agent_name=player_config.name,
    clock_now=clock.now,
    memory=mem,
    component_name="observation",
    timeframe=clock.get_step_size(),
    verbose=True
  )

  behav = tpb.Behaviour(
    "behaviour",
    model=model,
    memory=mem,
    player_config=player_config,
    num_behavs=5,
    verbose=True
  )

  attitude = tpb.Attitude(
    "attitude",
    model=model,
    memory=mem,
    player_config=player_config,
    components=[behav],
    verbose=True
  )

  people = tpb.People(
    "people",
    model=model,
    memory=mem,
    player_config=player_config,
    components=[behav],
    verbose=True
  )

  motiv = tpb.Motivation(
    "motivation",
    model=model,
    memory=mem,
    player_config=player_config,
    components=[people],
    verbose=True
  )

  norm = tpb.SubjectiveNorm(
    "norm",
    model=model,
    memory=mem,
    player_config=player_config,
    components=[motiv],
    verbose=True
  )

  thin_goal = tpb.ThinGoal(
    "thin_goal",
    model=model,
    memory=mem,
    player_config=player_config,
    verbose=True
  )

  full_model = tpb.TPB(
    "tpb",
    model=model,
    memory=mem,
    player_config=player_config,
    components=[attitude,norm,thin_goal],
    verbose=True
  )

  persona = generic_components.sequential.Sequential(
    name='persona',
    components=[
      behav,
      attitude,
      people,
      motiv,
      norm,
      full_model
    ]
  )

  situation = components.situation_perception.SituationPerception(
    name='situation',
    model=model,
    memory=mem,
    agent_name = player_config.name,
    clock_now=clock.now,
    verbose=True
  )

  plan = components.plan.SimPlan(
    model=model,
    memory=mem,
    agent_name=player_config.name,
    clock_now=clock.now,
    components=[situation],
    goal=full_model,
    time_adverb='5-minute',
    timescale='the next hour',
    verbose=True
  )

  full_plan = generic_components.sequential.Sequential(
    name='full_plan',
    components=[
      observation,
      persona,
      plan
    ]
  )

  agent = basic_agent.BasicAgent(
    agent_name=player_config.name,
    model=model,
    memory=mem,
    clock=clock,
    components=[full_plan],
    update_interval=datetime.timedelta(minutes=0),
    verbose=True
  )
  
  return agent

In [5]:
#@Build agents
NUM_PLAYERS = 2
agents: list[basic_agent.BasicAgent] = []
with concurrent.futures.ThreadPoolExecutor(max_workers=NUM_PLAYERS) as pool:
  for agent in pool.map(build_agent,
            player_configs[:NUM_PLAYERS]):
    agents.append(agent)

Alice's behaviour component has been fully activated.
Alice's attitude component has been fully activated.
Alice's people component has been fully activated.
Alice's motivation component has been fully activated.
Checking parameters to see if this function is being called and appropriately initialized.
Initialization: False
Alice's tpb component has been fully activated.
Using the thin goal...
Dorothy's behaviour component has been fully activated.
Dorothy's attitude component has been fully activated.
Dorothy's people component has been fully activated.
Dorothy's motivation component has been fully activated.
Checking parameters to see if this function is being called and appropriately initialized.
Initialization: False
Dorothy's tpb component has been fully activated.
Using the thin goal...
Alice wants to get a good rest tonight so that she can be well-prepared to help her grandmother all day tomorrow.Memories of Alice:
[03 Jul 1990 00:00:00] When Alice was 6 years old, she took apar

## GM Structure

In [6]:
game_master_memory = associative_memory.AssociativeMemory(
   sentence_embedder=embedder,
   importance=importance_model_gm.importance,
   clock=clock.now)

agent_list = [agent.name for agent in agents]

scenario_knowledge = generic_components.constant.ConstantComponent(
    state=shared_context,
    name='Background')

player_status = gm_components.player_status.PlayerStatus(
    clock_now=clock.now,
    model=model,
    memory=game_master_memory,
    player_names=agent_list,
    verbose=True)


convo_externality = gm_components.conversation.Conversation(
    players=agents,
    model=model,
    memory=game_master_memory,
    clock=clock,
    burner_memory_factory=blank_memory_factory,
    components=[player_status],
    cap_nonplayer_characters=3,
    shared_context=shared_context,
    verbose=True,
)

direct_effect_externality = gm_components.direct_effect.DirectEffect(
    players=agents,
    model=model,
    memory=game_master_memory,
    clock_now=clock.now,
    verbose=False,
    components=[player_status]
)

relevant_events = gm_components.relevant_events.RelevantEvents(
    clock.now, model, game_master_memory)
time_display = gm_components.time_display.TimeDisplay(clock)

In [7]:
# @title Create the game master object
env = game_master.GameMaster(
    model=model,
    memory=game_master_memory,
    clock=clock,
    players=agents,
    components=[
        scenario_knowledge,
        player_status,
        convo_externality,
        direct_effect_externality,
        relevant_events,
        time_display,
    ],
    randomise_initiative=False,
    player_observes_event=False,
    players_act_simultaneously=False,
    verbose=True,
)

## Scenario

In [ ]:
for premise in scenario_premise:
  game_master_memory.add(premise)
  for player in agents:
    player.observe(premise)
# @title Expect about 2-3 minutes per step.
episode_length = 1  # @param {type: 'integer'}
for _ in range(episode_length):
  CALL_TO_ACTION = (
    "Given the above, what will {agent_name} do for the "
    "next {timedelta} to achieve their current plan?"
  )
  env.step(action_spec = simulacrum_agent.ActionSpec(call_to_action=CALL_TO_ACTION,output_type="FREE"))

at Alice's doorstep, waiting for her.Events:
[01 Oct 2024 21:00:00] It is 9pm on a weeknight. Alice returns late from work and finds her friend Dorothy at her doorstep. 

The current time is: [01 Oct 2024 21:00:00]

Question: Given the above events and their time, what is the latest location of Dorothy and what are they doing?
Answer: Dorothy is at Alice's doorstep, waiting for her.

at Alice's doorstep, waiting for her.Events:
[01 Oct 2024 21:00:00] It is 9pm on a weeknight. Alice returns late from work and finds her friend Dorothy at her doorstep. 

The current time is: [01 Oct 2024 21:00:00]

Question: Given the above events and their time, what is the latest location of Dorothy and what are they doing?
Answer: Dorothy is at Alice's doorstep, waiting for her.

at Alice's doorstep, waiting for her.Events:
[01 Oct 2024 21:00:00] It is 9pm on a weeknight. Alice returns late from work and finds her friend Dorothy at her doorstep. 

The current time is: [01 Oct 2024 21:00:00]

Question: 

KeyboardInterrupt: 

4, Likelihood: 40)
2. **Dorothy feels indebted**: By accepting help from Alice, Dorothy might feel obligated to reciprocate in the future, potentially leading to an imbalance in their friendship. (Value: -3, Likelihood: 30)
3. **Alice says no**: Alice may decline to help, leaving Dorothy feeling embarrassed, disappointed, or even more stranded than before. (Value: -8, Likelihood: 20)Here are the potential consequences of Dorothy expressing frustration and anger towards Bob:

**Positive Consequences:**

1. **Catharsis**: Expressing her emotions could help Dorothy release tension and feel a sense of relief, allowing her to focus on finding an alternative solution for moving out tonight. (Value: 8, Likelihood: 80)
2. **Emotional Support**: Venting her frustrations to Alice might lead to emotional support and encouragement from her friend, helping Dorothy feel less alone in this difficult situation. (Value: 7, Likelihood: 90)
3. **Motivation**: Expressing anger towards Bob could motivate D

Error in StreamingStdOutCallbackHandler.on_llm_new_token callback: ZMQError('Socket operation on non-socket')
--- Logging error ---
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniconda/base/envs/concordia/lib/python3.11/site-packages/langchain_core/callbacks/manager.py", line 231, in handle_event
    event = getattr(handler, event_name)(*args, **kwargs)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Caskroom/miniconda/base/envs/concordia/lib/python3.11/site-packages/langchain_core/callbacks/streaming_stdout.py", line 33, in on_llm_new_token
    sys.stdout.write(token)
  File "/opt/homebrew/Caskroom/miniconda/base/envs/concordia/lib/python3.11/site-packages/ipykernel/iostream.py", line 662, in write
    self._schedule_flush()
  File "/opt/homebrew/Caskroom/miniconda/base/envs/concordia/lib/python3.11/site-packages/ipykernel/iostream.py", line 559, in _schedule_flush
    self.pub_thread.schedule(_schedule_in_thread)
  File "/opt/hom

 are 5 potential behaviors that Alice can take in response to the situation:

1. **Agree to help Dorothy immediately**: Despite being exhausted after work and having commitments with her grandmother the next day, Alice's sense of responsibility and empathy may lead her to agree to help Dorothy move out of her house tonight.

2. **Offer alternative solutions**: Alice might suggest alternative solutions that don't require her to help with the moving tonight, such as helping Dorothy find a last-minute moving service or postponing the move to another day when she's more available.

3. **Ask for clarification and details**: Before committing to anything, Alice may ask Dorothy for more information about the situation, such as why Bob didn't show up and what exactly needs to be moved out of the house.

4. **Express her concerns and limitations**: Alice might explain her own situation to Dorothy, expressing her exhaustion after work and her commitments with her grandmother the next day, and se

### Things to add

- Summarize deliberation on the potential actions, including thoughts on the possible positive and negative consequences (counterfactuals should remain throughout the whole)
- 